# Whole Slide Images

Programme de ce TP:
- Manipuler des lames histologiqes numérisées (ci-après WSI) avec OpenSlide
- Extraire des _patches_ de WSI
- Comprendre le concept de teinte et de normalisation

![img1](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41416-020-01122-x/MediaObjects/41416_2020_1122_Fig1_HTML.png?as=webp "Source: https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41416-020-01122-x/MediaObjects/41416_2020_1122_Fig1_HTML.png?as=webp")

Source: Echle et al., Deep learning in cancer pathology: a new generation of clinical biomarkers

# Prérequis pour ouvrir le notebook (liste diffusée par mail)

- 0) Créer un compte Google pour avoir accès à Google Drive.
- 1) Ouvrir ce lien de partage : https://drive.google.com/drive/folders/1j-XK2umdsMwAPsOgEgqt4rR56q6oBGSf?usp=sharing
- 2) Dans l'en-tête "Partagés avec moi", aller dans "data", et cliquer sur "ajouter à mon drive".
- 3) Pour ouvrir les TP, aller d'abord dans https://colab.research.google.com/
- 4) Sur la page de chargement (figure ci-dessous): cliquer sur "GitHub", puis rentrer le lien https://github.com/afiliot/TPDUIA ; vous pouvez refuser les demandes d'autorisation. Puis, dans le menu déroulant des fichiers, sélectionnez `TPDUIA/2022/whole_slide_images.ipynb`

# Prérequis après ouverture du notebook

- 5) Cliquer dans le menu sur `Affichage` puis `Dérouler les rubriques`
- 6) Réaliser un point de montage avec le contenu de son drive pour accéder aux données du TP (cliquez sur la cellule ci-dessous).
- 7) Une fois le notebook ouvert, cliquer sur `Modifier` puis `Paramètres du notebook` puis `accélérateur matériel`, et finalement `GPU` pour disposer d'une carte graphique virtualisée.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# Données utilisées: programme TCGA https://portal.gdc.cancer.gov/

Lames issues de la cohorte TCGA-STAD (n=443 pour 442 WSI) recensant des patients atteints d'adénocarcinome gastrique (cancer de l'estomac), <a href="https://portal.gdc.cancer.gov/projects/TCGA-STAD"> disponible ici (page d'accueil)<a>.
    
TCGA: The Cancer Genome Atlas, est un programme de recherche tourné vers la caractérisation moléculaire d'un très large panel de cancers (33 types différents pour 20 000 patients inclus). Le portail TCGA est un recueil extrêmement précieux de données -omiques et d'échantillons biologiques numérisés. Ce programme a démarré en 2006 à l'initiative du NCI (National Cancer Institue, US), du NHI (National Institutes for Health, US) et du NHGR (National Human Genome Research Institute). Les données TCGA sont ouvertes à la communauté scientifique et sont aujourd'hui utilisées mondialement dans le monde de la recherche académique.
   
Le portail TCGA propose un certain nombre d'API et web service, comme ces:
- <a href="https://portal.gdc.cancer.gov/analysis?analysisId=2022-03-01T11%3A15%3A31.371Z&analysisTableTab=result"> outil de visualisation des données cliniques<a>
- <a href="https://portal.gdc.cancer.gov/image-viewer?caseId=e52396d1-acd6-44f4-99c5-1a367128008d&filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.case_id%22%2C%22value%22%3A%5B%22set_id%3AezowRX8Bl4ySVQYe--M_%22%5D%7D%7D%5D%7D&selectedId=4e52fa82-b5c8-4432-9149-5ebd3ada8e0c"> outil de visualisation des lames<a>

# Consultation des lames sur le portail TCGA

- Liste des lames "diagnostiques" de la cohorte TCGA-STAD, _i.e._ des lames issues de prélévements opératoires <a href="https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22content%22%3A%7B%22field%22%3A%22cases.project.project_id%22%2C%22value%22%3A%5B%22TCGA-STAD%22%5D%7D%2C%22op%22%3A%22in%22%7D%2C%7B%22content%22%3A%7B%22field%22%3A%22files.data_format%22%2C%22value%22%3A%5B%22svs%22%5D%7D%2C%22op%22%3A%22in%22%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.experimental_strategy%22%2C%22value%22%3A%5B%22Diagnostic%20Slide%22%5D%7D%7D%5D%7D"> disponible sur le portail TCGA <a>.

- Il est également possible de consulter les données par patients <a href="https://portal.gdc.cancer.gov/exploration?filters=%7B%22content%22%3A%5B%7B%22content%22%3A%7B%22field%22%3A%22cases.case_id%22%2C%22value%22%3A%5B%22set_id%3A6D8nRX8BEVoSrQHWcDxr%22%5D%7D%2C%22op%22%3A%22IN%22%7D%5D%2C%22op%22%3A%22AND%22%7D"> ici<a>.
    
- Les lames sont marquées par "DX1" ou "DX2".

______________
On va ici appliquer des filtres à la cohorte:
- 1) Patient de sexe masculin,
- 2) Patient décédé,
- 3) Patient atteint d'un adénocarcinome de l'estomac,
- 4) Patient donc le stade tumoral T (TNM) est égal à 4b

**Ceci permet d'aboutir à une liste de 6 patients pour 6 lames,** <a href="https://portal.gdc.cancer.gov/image-viewer?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.case_id%22%2C%22value%22%3A%5B%22set_id%3A_myuY38BEqg9E8t96O1c%22%5D%7D%7D%5D%7D"> **disponible ici**<a>.
______________
Explication des différents stades <a href="https://cancer.ca/fr/cancer-information/cancer-types/stomach/staging"> selon l'AJCC <a>.
    
Une tumeur T4b est incluse dans la catégorie AJCC IIIC décrivant les cas suivants (1 ou 2):
- La tumeur a envahi la sous-séreuse ou elle a traversé la séreuse. Le cancer s’est aussi propagé à au moins 16 ganglions lymphatiques situés près de l’estomac
- La tumeur a envahi des régions ou des organes voisins. Le cancer s’est aussi propagé à au moins 7 ganglions lymphatiques situés près de l’estomac

______________

![img1](https://cdn.cancer.ca/-/media/cams/stomach/f72b40a0-b64f-11ea-bc3d-0242df4d59be-fr.png?h=375&mw=1586&w=454&rev=d0a96a936a96477aad98a5ffd624081d&extension=webp&hash=025B40D2FF66F1C317112406156877FD "Source: https://cancer.ca/fr/cancer-information/cancer-types/stomach/staging")

Source: https://cancer.ca/fr/cancer-information/cancer-types/stomach/staging

![img1](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.revmed.ch%2Fvar%2Fsite%2Fstorage%2Fimages%2Frms-2422%2Fimages%2F22736_1.gif%2F468899-1-fre-CH%2F22736_1.gif_i770.gif&f=1&nofb=1 "Source https://www.revmed.ch/")

Source: https://www.revmed.ch/

# Installer open-slide

OpenSlide Python est une interface Python de la librairie OpenSlide https://openslide.org/ qui gère les formats suivants:
- Aperio (.svs, .tif)
- Hamamatsu (.vms, .vmu, .ndpi)
- Leica (.scn)
- MIRAX (.mrxs)
- Philips (.tiff)
- Sakura (.svslide)
- Trestle (.tif)
- Ventana (.bif, .tif)
- Generic tiled TIFF (.tif)

OpenSlide est une librairie C proposant une interface simple, open-source pour la lecture et la manipulation de WSI, caractérisées par une structure pyramidale contenant des niveaux de très haute résolution.

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python
!pip install p-tqdm
!git clone https://github.com/openslide/openslide-python.git
!mv openslide-python openslidepython
!wget https://raw.githubusercontent.com/afiliot/TPDUIA/main/TPDUIA/2022/scripts.py
!mv scripts.py /content/gdrive/MyDrive/TPDUIA/

# Ajout du code `scripts.py` dans Colab

In [ ]:
import sys
sys.path.insert(0, '/content/gdrive/My Drive/TPDUIA/')

# Import des librairies nécessaires

In [ ]:
# librairies utilitaires
import os
from pprint import pprint
from glob import glob
from random import sample

# numpy pour les matrics
import numpy as np

# visualisation de la progression des processus
from tqdm import tqdm
from p_tqdm import p_map

# manipulation et traitement des WSI
import openslide

# visualisation
from PIL import Image

# Chemin vers les données

(exécutez cette cellule)

In [ ]:
wsi_folder = '/data/freganet/RAW/TCGA/WSI/' #'/content/gdrive/MyDrive/TPDUIA/SLIDES//'

# Chargement des lames

Nous allons travailler avec les 6 lames précédentes issues de 6 patients différents:

    '8362/TCGA-BR-8362-01Z-00-DX1.178c994d-2cf5-4385-a807-fc08e0cf10f6.svs', # patient 8362
    'A4CS/TCGA-BR-A4CS-01Z-00-DX1.84B5892C-6C7F-474D-BA09-04E870A47764.svs', # patient A4CS
    'A91D/TCGA-VQ-A91D-01Z-00-DX1.B7E99494-FA16-42B5-AC1C-1498A7989111.svs', # patient A91D
    'A4GQ/TCGA-HU-A4GQ-01Z-00-DX1.699A0E57-FE02-4329-9826-3EFC2DECD408.svs', # patient A4GQ
    '8686/TCGA-BR-8686-01Z-00-DX1.8f559dd9-228e-4ed2-a7e4-16e04b80f05c.svs', # patient 8686
    '8590/TCGA-BR-8590-01Z-00-DX1.504afa1d-3a8f-423f-9edf-4df35720c05f.svs'  # patient 8590

### Visualisation d'une lame avec ``deepzoom_server.py``

Executez le code ci-dessous.

In [ ]:
wsi_path = os.path.join(
    wsi_folder,
    'A4CS/TCGA-BR-A4CS-01Z-00-DX1.84B5892C-6C7F-474D-BA09-04E870A47764.svs'
)

Pour lancer un serveur de visualisation sur le port 5000, décommentez et exécutez le code ci-dessus.

In [ ]:
from scripts import launch_server

# lancement d'un serveur sur le port 5000
launch_server(wsi_path)

Puis connectez vous à http://127.0.0.1:5000/ pour visualiser la lame.

Une fois la visualisation terminée, vous pouvez cliquer sur l'icône d'interruption du notebook. La cellule précédente devra afficher "Server stopped" une fois le notebook interrompu.

In [ ]:
# liste des chemins correspondant aux lames de 0 à 5
wsi_paths = [
    
    '8362/TCGA-BR-8362-01Z-00-DX1.178c994d-2cf5-4385-a807-fc08e0cf10f6.svs', # patient 8362, lame 0
    'A4CS/TCGA-BR-A4CS-01Z-00-DX1.84B5892C-6C7F-474D-BA09-04E870A47764.svs', # patient A4CS, lame 1
    'A91D/TCGA-VQ-A91D-01Z-00-DX1.B7E99494-FA16-42B5-AC1C-1498A7989111.svs', # patient A91D, lame 2
    'A4GQ/TCGA-HU-A4GQ-01Z-00-DX1.699A0E57-FE02-4329-9826-3EFC2DECD408.svs', # patient A4GQ, lame 3
    '8686/TCGA-BR-8686-01Z-00-DX1.8f559dd9-228e-4ed2-a7e4-16e04b80f05c.svs', # patient 8686, lame 4
    '8590/TCGA-BR-8590-01Z-00-DX1.504afa1d-3a8f-423f-9edf-4df35720c05f.svs'  # patient 8590, lame 5
    
]

In [ ]:
# vous pouvez explorer d'autres lames
# 1) exécutez la cellule et entrez un numéro de lame entre 0 et 5
numero_lame = 1
assert numero_lame in range(6), 'Entrez un chiffre entre 0 et 5!'

# chemin de la lame d'intérêt que vous avez sélectionnée
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])

# ouverture du serveur, n'oubliez pas d'aller sur http://127.0.0.1:5000/ pour visualiser la lame
launch_server(wsi_path)

## Utilisation de l'objet ``OpenSlide`` pour charger une lame

In [ ]:
wsi_folder = '/data/freganet/RAW/TCGA/WSI/' #'/content/gdrive/MyDrive/TPDUIA/SLIDES//'
wsi_path = os.path.join(
    wsi_folder,
    'A4CS/TCGA-BR-A4CS-01Z-00-DX1.84B5892C-6C7F-474D-BA09-04E870A47764.svs'
)
wsi = openslide.OpenSlide(wsi_path)

### Visualisation des méta-données

In [ ]:
wsi_metadata = dict(wsi.properties)
aperio_metadata = {
    k: v for (k, v) in wsi_metadata.items() if k.startswith('aperio')
}
openslide_metadata = {
    k: v for (k, v) in wsi_metadata.items() if k.startswith('openslide')
}

### Sur la numérisation

In [ ]:
pprint(aperio_metadata)

### Sur la lame elle-même

In [ ]:
pprint(openslide_metadata)

### Sélection des informations utiles

In [ ]:
from scripts import print_slide_details

print_slide_details(wsi, show_thumbnail=True)

In [ ]:
# vous pouvez changer de lame
numero_lame = 1
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print_slide_details(wsi, show_thumbnail=True)

## Sélection de zones d'intérêt

In [ ]:
from scripts import display_region

x = 18500
y = 19500
level = 2
width = 500
height = 500
display_region(wsi, x, y, level, width, height)

In [ ]:
x = 18000
y = 3000
level = 2
width = 500
height = 500
display_region(x, y, level, width, height)

# Séparation du tissu et du fond blanc

### Technique d'Otsu

La technique d'Otsu est une technique de seuillage permettant de séparer une image en plusieurs classes ou groupes d'intensité. Cette technique minime la variance intra-classe des groupes construits comme étant homogènes (vis-à-vis de seuils d'intensité).

![img1](https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.meccanismocomplesso.org%2Fwp-content%2Fuploads%2F2017%2F04%2FOpenCV-Otsu-Binary-threshold-of-noisy_leaf-image-2-1.jpg&f=1&nofb=1 "Source: https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.meccanismocomplesso.org%2Fwp-content%2Fuploads%2F2017%2F04%2FOpenCV-Otsu-Binary-threshold-of-noisy_leaf-image-2-1.jpg&f=1&nofb=1")

Source: https://www.meccanismocomplesso.org/opencv-python-otsu-binarization-thresholding/


### Conversion de l'image RGB vers l'espace de couleur LAB

_Explications partiellement empruntées de l'excellente page Wikipédia sur le sujet: https://fr.wikipedia.org/wiki/L*a*b*_CIE_1976_

Pour séparer les tissus du fond blanc de lame, l'approche d'Otsu est la plus populaire en pathologie digitale. En pratique, cette séparation ne s'effectue pas dans le domaine RGB (Red-Green-Blue) standard mais l'espace chromatique LAB.

L'espace chromatique LAB CIE 1976, ou CIELAB, est utilisé pour la caractérisation des couleurs de surface. Il a été introduit par la Commission Internationale de l'Eclairage en 1976.

Les coordonnnées cartésiennes (L, a, b) peuvent être calculées à partir des coordonnées (r, g, b) dans l'espace RGB. La méthode de calcul est décrite ici: https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_lab

Si l'on regarde le diagramme ci-dessous, l'espace CIE LAB peut être assimilé à une sphère aplatie au sommets (sur l'axe a). De telle sorte que:
- l'axe vertical L correspond à la clarté ou luminosité allant de 0 à 100 (0 pour le noir où l'absorption est totale, et 100 pour le blanc où la réflexion est totale)
- dans chaque plan horizontal de cette sphère (sur l'axe b), on trouve deux axes orthonormés:
    - -a vers +a qui correspond à l'axe vert-rouge (rougissement) 
    - -b vers b qui correspond à l'axe bleu-jaune (jaunissement)
    
La teinte et la saturation d'une couleur donnée seront déterminées par les coordonnées cartésiennes a, b. Plus on s'éloigne du centre (où la couleur est neutre), plus la couleur est saturée.
  
![img1](https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.laelith.fr%2FCours%2FIllus%2F010-cielab_3D.jpg&f=1&nofb=1 "Source: https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.laelith.fr%2FCours%2FIllus%2F010-cielab_3D.jpg&f=1&nofb=1")



Illustrations avec une image RGB converties dans l'espace LAB dont les canaux de couleur ont été séparés:

![img1](https://miro.medium.com/max/1050/1*mJQuHRG9La6hZNV4B8hP8A.png "Source: https://miro.medium.com/max/1050/1*mJQuHRG9La6hZNV4B8hP8A.png")

![img1](https://miro.medium.com/max/1050/1*IfZrdyN-pe1oKbfBEPOM1w.png "Source: https://miro.medium.com/max/1050/1*IfZrdyN-pe1oKbfBEPOM1w.png")


Sources:
- https://fr.wikipedia.org/wiki/L*a*b*_CIE_1976
- https://www.universalis.fr/encyclopedie/colorimetrie/3-l-espace-cielab-1976-ou-l-a-b/#DE041512
- https://towardsdatascience.com/understand-and-visualize-color-spaces-to-improve-your-machine-learning-and-deep-learning-models-4ece80108526

In [ ]:
from scripts import segment_region

_ = segment_region(wsi, plot='all channels')

In [ ]:
# vous pouvez changer de lame
numero_lame = 2
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
lab, img_raw, img_new, mask = segment_region(wsi, plot='lab only')

In [ ]:
# vous pouvez changer de lame
numero_lame = 4
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
lab, img_raw, img_new, mask = segment_region(wsi, plot='lab only')

# Création des patches

Cette étape est indispensable à l'entraînement d'algorithmes de machine learning ou de deep learning. Elle consiste à créer des _patches_ (ou imagettes, en anglais _tiles_) qui vont quadriller la lame entière. 

Cette étape prend en compte un certain nombre de paramètres _a priori_, que nous allons fixer de la manière suivante:
- taille physique des patches: 256$\mu m$ par 256$\mu m$ 
- niveau de résolution: 40x ou 0.25$\mu mpp$
- taille (en pixels) des patches: 256$px$ par 256 $px$
- _overlap_ (ou taux de superposition): 0%
- critère de sélection et stockage d'un patch: % fond blanc < 20%

![img1](https://www.pixelscientia.com/images/wsi_weak_supervision1.jpg "Source: https://www.pixelscientia.com/images/wsi_weak_supervision1.jpg")


Pour réaliser cette étape, nous allons utiliser l'objet `DeepZoomGenerator` du package `openslide`. Cet objet permet d'accéder aux régions d'une lame selon plusieurs niveaux de résolution, de manière optimisée. Sa syntaxe est spécifique et nécessite une bonne compréhension de la documentation.  

In [ ]:
# paramètres
tile_size = 256 # micromètres
target_shape = 256 # pixels
overlap = 0
threshold_background_max = 0.2

On créée l'objet DZG en lui ajoutant un certain nombre d'attributs.

Cellules à exécuter: 1) chargement de la lame, 2) création de l'objet DZG.

In [ ]:
numero_lame = 4
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
wsi.get_thumbnail((200, 200))

In [ ]:
from scripts import create_dzg, print_dz_properties

dz = create_dzg(wsi, wsi_path, tile_size, target_shape, overlap, threshold_background_max)

On peut maintenant s'intéresser à l'objet `dz` et certains de ces attributs.

In [ ]:
print_dz_properties(dz)

On va désormais utiliser plusieurs fonctions utilitaires pour extraire les patchs de la lame.

Commençons par créer une grille de toutes les coordonnées des patchs.

In [ ]:
from scripts import get_grid

grid = get_grid(dz)
print('Premières coordonnées');print(grid[:10])
print('Dernières coordonnées');print(grid[-5:])
print('Nombre de patches');print(grid[-1])

On peut utiliser la fonction `get_regions` pour mieux comprendre son fonctionnement.

In [ ]:
from scripts import get_regions

get_regions(dz, 0, 50)

Et enfin utiliser la fonction `ij_tiling` pour visualiser le résultat de la segmentation sur le patch de coordonnées (10, 51). Attention: (10, 51) signifie 10ième colonne, et 51ième ligne.

In [ ]:
from scripts import ij_tiling
# (10, 51): 10ième colonne, 51ième ligne
tile, tile_path = ij_tiling(dz, (10, 51), plot=True)

On peut également afficher l'ensemble des patchs sur une même ligne.

In [ ]:
from scripts import plot_tiles_on_row

plot_tiles_on_row(dz, grid, j=51)

Vous pouvez essayer de retrouver l'imagette (51, 0) ou (51, 1) sur la lame.

In [ ]:
#launch_server(wsi_path)

On peut retrouver la bande correspondante à la ligne 40 grâce au script suivant.

In [ ]:
from scripts import visualize_band

row = 51
visualize_band(wsi, dz, delta=-400, row=row)

On change le niveau de résolution ! 

In [ ]:
visualize_band(wsi, dz, row=row, delta=-800, band_level=0)

Vous pouvez désormais essayer avec d'autres lignes de la lame.

In [ ]:
row = 48

visualize_band(wsi, dz, row-1, band_level=2)
visualize_band(wsi, dz, row, band_level=1)
visualize_band(wsi, dz, row, band_level=0)

Pour finir sur la visualisation, nous allons observer le résultat de la segmentation sur certains patchs tirés aléatoirement sur la lame (ré-exécuter le code jusqu'à obtenir quelques images).

In [ ]:
for coords in tqdm(sample(grid, k=20)):
    _ = ij_tiling(dz, coords, plot=True)

Nous allons maintenant stocker toutes les images dans le drive dans une taille plus grande (512 micromètres) pour accélerer le processus.

In [ ]:
tile_size = 512 # micromètres
threshold_background_max = 0.8
target_shape = 224
overlap = 0
dz = create_dzg(wsi, wsi_path, tile_size, target_shape, overlap, threshold_background_max)
tiles_names = os.listdir(dz.output_folder)
grid = get_grid(dz)
print(f'Total: {len(grid)} patches to process.')

In [ ]:
print_dz_properties(dz)

On peut paralléliser la fonction ij_tiling sur toutes les coordonnées de la grille.

In [ ]:
print('Running...')
_ = p_map(lambda x: ij_tiling(dz, x, plot=False), grid)
print(f'Done. Stored {len(os.listdir(dz.output_folder))} patches.')

Et désormais afficher le résultat de la segmentation !

In [ ]:
from scripts import plot_patch_mask

plot_patch_mask(dz)

Vous pouvez vous exercer avec d'autres lames et d'autres paramètres.

-> Cellule pour le chargement de la lame.

In [ ]:
numero_lame = 3
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
wsi.get_thumbnail((200, 200))

-> Cellule pour la création de l'objet DZG (+ segmentation).

In [ ]:
tile_size = 256 # micromètres
target_shape = 224 # pixels
overlap = 0
threshold_background_max = 0.4
scale_factor = 1/64 if numero_lame in [0, 2] else 1/32

In [ ]:
dz = create_dzg(wsi, wsi_path, tile_size, target_shape, overlap, threshold_background_max, scale_factor)
tiles_names = os.listdir(dz.output_folder)
grid = get_grid(dz)
print(f'Total: {len(grid)} patches to process.')

-> Cellule pour le stockage des patches.

In [ ]:
print('Running...')
_ = p_map(lambda x: ij_tiling(dz, x, plot=False), grid)
print(f'Done. Stored {len(os.listdir(dz.output_folder))} patches.')

-> Cellule pour l'affichage de la mosaïque.

In [ ]:
plot_patch_mask(dz)

# Teinte et normalisation 

_La coloration HE_

La coloration à l'hématoxyline et à l'éosine (abrégée par coloration HE) est une technique de coloration bichromatique utilisée en routine en histopathologie. La coloration de routine permet au pathologiste de se faire une idée de la pathologie présente et d’avoir une vision globale de la morphologie des tissus (noyau, cytoplasme et collagène).
Il peut ensuite demander à revoir le même tissu mais cette fois avec un élément tissulaire particulier qui aura été mis en évidence de façon spécifique via des colorations spéciales.

La coloration HE se compose:
- d'un colorant nucléaire, l'hématoxyline: colorant cationique (ou basique) qui a une affinité pour les éléments cellulaires chargés négativement (= anioniques ou acides) dits basophiles
- et d'un colorant cytoplasmique, l'éosine: colorant anionique (ou acide), qui a une affinité pour les éléments cellulaires chargés positivement (= cationiques ou basiques) dits éosinophiles.

Ces deux composants colorent:
- pour l'hématoxyline: le cytoplasme en rose et les autres éléments cellulaires basiques en rose/rouge plus ou moins vifs selon leur acidophilie (affinité pour les colorants acides)
- pour l'éosine: les noyaux en bleu/violet en se fixant sur les acides nucléiques.

![img1](https://upload.wikimedia.org/wikipedia/commons/8/86/Emphysema_H_and_E.jpg "Source: https://upload.wikimedia.org/wikipedia/commons/8/86/Emphysema_H_and_E.jpg")

Source: https://fr.wikipedia.org/wiki/Coloration_%C3%A0_l%27h%C3%A9matoxyline_et_%C3%A0_l%27%C3%A9osine

En microscopie optique, on observe également grâce à la coloration HE:
- Les basophiles en pourpre
- Les muscles en rose foncé
- Les globules rouges en rouge cerise
- Le collagène en rose pâle


*HED et IHC: deux termes à connaître en histopathologie*

La coloration HED est particulièrement utilisée en immunohistochimie (IHC), _la_ méthode la plus utilisée pour le diagnostic de cancer à l'heure actuelle. L'IHC permet de mettre en évidence certains marqueurs moléculaires caractéristiques de cellules ou d'événements anormaux tels que la prolifération de cellules cancéreuses ou la mort cellulaire, par exemple CD10 (CALLA) qui marque les cellules de l'adénocarcinome du rein (ou encore CD117 pour les cellules tumorales gastro-intestinales, pour plus de détails: https://en.wikipedia.org/wiki/Immunohistochemistry).

L'IHC est aussi largement utilisée en recherche fondamentale pour comprendre la distribution et la localisation de biomarqueurs et de protéines différentiellement exprimés dans les différentes parties d'un tissu biologique


D'un point de vue pratique, le prélévement de tissu est placé sur un support solide et un anticorps est ajouté à la préparation, fixant spécifiquement la protéine à détecter. Plus tard, l'échantillon découpé est plongé dans un bain de Diaminobenzidine permettant de visualiser l'anticorps fixé. Les noyaux, le cytoplasme, les membranes sont ensuite révélées par contre-coloration à l'Hématoxyline et / ou à l'Eosine. `scikit-image` propose de visualiser les 3 canaux différents, H, E et DAB à partir d'une image normale RGB.

_Exemple_

Sur cette image, un échantillon a été coloré grâce au marquage PIN-4, qui est "cocktail" de différents anti-corps ciblant les protéines p63, CK-5, CK-14 et P504S et permettant de distinguer les glandes bénignes de cellules tumorales de la prostate. Sur la gauche, on note ainsi une glande bénigne, tandis qu'à droite, une cellule tumorale. Cette cellule ne présente pas de celulles épithéliales basales (marquées en marron par p63, CK-5 et CK-14), contrairement à la cellule saine. En plus, grâce au marquage PIN-4, les cellules tumorales présentent des cytoplasmes rouges (marqués par P504S).

![img1](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/PIN-4_staining_of_benign_prostate_gland_and_adenocarcinoma.jpg/400px-PIN-4_staining_of_benign_prostate_gland_and_adenocarcinoma.jpg "Source: https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/PIN-4_staining_of_benign_prostate_gland_and_adenocarcinoma.jpg/1024px-PIN-4_staining_of_benign_prostate_gland_and_adenocarcinoma.jpg")

Source: https://en.wikipedia.org/wiki/Immunohistochemistry

_Du RGB au HED_

La librairie `scikit-image` permet de convertir une image RGB en une image "HED" dans laquelle les 3 canaux de couleur correspondent aux colorations Hématoxyline, Eosine et Diaminobenzidine respectivement. Cette méthode de conversion a été proposée en 2001 par Ruifrok et Johnston (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4354574/). Dans un premier temps, les auteurs ont d'abord usé de la conversion de RGB vers l'espace CIE LAB puis l'application de 3 filtres distincts: un filtre "rouge" pour l'Eosine, un filtre "bleu" pour l'Hématoxyline et un filtre "marron" pour la Diaminobenzidine. L'ensemble de ces étapes a permis - par tatônnement - d'aboutir à des résultats satisfaisants. Une fois l'algorithme validé, les étapes décrites ci-dessus ont été synthétisées sous la forme d'une opération de (dé)convolution*:

$$
\left[\begin{array}{l}
R \\
G \\
B
\end{array}\right]=\left[\begin{array}{lll}
0.65 & 0.70 & 0.29 \\
0.07 & 0.99 & 0.11 \\
0.27 & 0.57 & 0.78
\end{array}\right] \, \times \left[\begin{array}{l}
H \\
E \\
D
\end{array}\right]
$$

ou à l'inverse:

$$
\left[\begin{array}{l}
H \\
E \\
D
\end{array}\right]=\left[\begin{array}{lll}
1.878 & -1.008 & -0.556 \\
-0.066 &  1.135 & -0.136 \\
-0.602 & -0.48 &  1.574
\end{array}\right] \, \times \left[\begin{array}{l}
R \\
G \\
B
\end{array}\right]
$$


Ce qui donne en termes d'implémentation dans `scikit-image`:

    rgb_from_hed = np.array([[0.65, 0.70, 0.29],
                             [0.07, 0.99, 0.11],
                             [0.27, 0.57, 0.78]])
                             
    hed_from_rgb = linalg.inv(rgb_from_hed)

*Comprendre la convolution grâce à la visualisation: https://setosa.io/ev/image-kernels/

Note: code _ad-hoc_ pour obtenir l'Hématoxyline (plus basique mais plus explicite que le code `scikit-image`)

```
rgb_from_hed = np.array([[0.65, 0.70, 0.29],
                         [0.07, 0.99, 0.11],
                         [0.27, 0.57, 0.78]])

hed_from_rgb = np.linalg.inv(rgb_from_hed)
tile = np.array(Image.open('/chemin/vers/le/patch.png'))
a = tile @ hed_from_rgb # c'est l'opération de convolution !
plt.imshow(255-a[..., 0], cmap='Blues')
```

## Extraction des teintes H, E et DAB

On reprend la lame numéro 2.

In [ ]:
numero_lame = 2
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
wsi.get_thumbnail((400, 400))

Et on stocke les chemins vers les patches de cette lame.

In [ ]:
patches = glob('/data/freganet/TPDU/A4CS_taille-256/*.png')

On affiche les images.

In [ ]:
from scripts import render_hed

# visualisation
for patch in sample(patches, k=10):
    render_hed(patch)

## Normalisation des teintes

Avant toute chose, il vous faut installer les packages `staintools` et `spams`. `staintools` est une librairie Python proposant de nombreuses méthodes de normalisation des teintes d'images histologiques.

In [ ]:
!pip install staintools
!pip install spams

La méthode de normalisation la plus utilisée en pathologie digitale est la méthode de normalisation de Macenko (implémentation disponible <a href=https://staintools.readthedocs.io/en/latest/_modules/staintools/normalization/macenko.html>ici<a/>). Mais il existe aussi d'autres méthodes comme celle de Reinhard ou de Vadahane.

M. Macenko et al., "A method for normalizing histology slides for quantitative analysis," 2009 IEEE International Symposium on Biomedical Imaging: From Nano to Macro, 2009, pp. 1107-1110, doi: 10.1109/ISBI.2009.5193250.

    M. Macenko et al., "A method for normalizing histology slides for quantitative analysis," 2009 IEEE International Symposium on Biomedical Imaging: From Nano to Macro, 2009, pp. 1107-1110, doi: 10.1109/ISBI.2009.5193250.
    
Normaliser les patches avant une tâche d'entraînement (classification, segmentation, synthèse, etc.) permet, selon le cas d'application, d'augmenter les performances prédictives des modèles sous-jacents et leur capacité à généraliser à des nouvelles données (normalisées et donc plus homogènes). Ceci permettrait de palier au fameux _batch effect_: certains facteurs exogènes, techniques et/ou technologiques (en particulier de numérisation), deviennent des facteurs confondants, perturbants les résultats d'une expérience par le biais de corrélation (par exemple, les modèles performent mal sur les données du centre A car la technique de numérisation n'est pas la même que celle du centre B utilisé pour l'entraînement).

Néanmoins, le bénéfice de la normalisation n'est pas systématique. L'équipe de Kather et al., pinoière dans le domaine de la pathologie digitale appliquée à la découverte de biomarqueurs moléculaires et pronostics du cancer du côlon, utilise systématiquement la normalisation. Dans leur étude d'octobre 2020, "Clinical-Grade Detection of Microsatellite Instability in Colorectal Tumors by Deep Learning", les auteurs montrent une légère amélioration des résultats sur une cohorte de validation externe de 771 patients (cohorte d'entraînement de 6604 patients):

    Native (non-normalized) image tiles (Figure A) were subjectively more diverse in terms of staining hue and intensity than normalized tiles (Figure B). [...] To test if color normalization improves the external test performance of MSI and dMMR predictors, we repeated experiment 4 [...] after color normalization. In this case, AUROC did improve (0.96 [0.93–0.98] vs. 0.95 [0.92–0.96]). This slight increase in AUROC translated into a higher specificity at predefined sensitivity levels, reaching 58% specificity at 99% sensitivity. These data show that color normalization can further improve classifier performance and improve generalizability of deep learning–based inference of MSI and dMMR status.
    
![img1](https://ars.els-cdn.com/content/image/1-s2.0-S0016508520348186-gr3.jpg "Source: https://ars.els-cdn.com/content/image/1-s2.0-S0016508520348186-gr3.jpg")  
  
Source: https://www.sciencedirect.com/science/article/pii/S0016508520348186?via%3Dihub

Un des articles les plus "à jour" sur les techniques de normalisation est celui de Salehi et al. (2021): https://arxiv.org/abs/2002.00647. Les auteurs comparent différentes méthodes de normalisation standard à leur méthode, qui consiste en un GAN dit cyclique (ou Cyclic-GAN) dont le rôle est de retrouver les images de référence à partir d'images dérivées aux teintes modifiées.

### La normalisation en pratique

#### Définition d'une image de référence, `target_slide`

On définit une lame comme l'image de référence dont il faudra reproduire la teinte.

In [ ]:
from scripts import wsi_to_numpy

# vous pouvez choisir un numéro de lame entre 0 et 5
numero_lame = 2
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
target_wsi = openslide.OpenSlide(wsi_path)
# on stocke l'image à un niveau de résolution faible
# pour la passer en mémoire
target_img = wsi_to_numpy(target_wsi, level=3)
print(f'Image with shape: {target_img.shape[:-1]}')
# on récupère également tous les patches associés
# à la lame, grâce aux opérations précédentes
target_patches = glob('/data/freganet/TPDU/A91D_taille-256/*.png')
# enfin, on affiche une macro
wsi.get_thumbnail((200, 200))

On définit maintenant les patches _source_, ceux dont on va modifier les teintes pour les aligner sur celles de la lame de référence.

In [ ]:
# là aussi, possibilité de modifier entre 0 et 5 (éviter le même numéro que précédemment)
numero_lame = 1
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
source_wsi = openslide.OpenSlide(wsi_path)
# stockage des chemins vers les patches
source_patches = glob('/data/freganet/TPDU/A4CS_taille-256/*.png')
# macro
wsi.get_thumbnail((200, 200))

On utilise le package `staintools`.

In [ ]:
import staintools

# standardisation de la luminosité pour améliorer l'extraction des tissus
target_img = staintools.LuminosityStandardizer.standardize(target_img)
# on estime les paramètres de la méthode Macenko qui serviront
# ensuite à normaliser de nouveaux patches
normalizer = staintools.StainNormalizer(method='macenko')  #ou reinhard, rahadane, etc.
normalizer.fit(target_img)

On applique la normalisation à un sous-ensemble de patch de la lame source.

In [ ]:
for s in sample(source_patches, 10):
    # ouverture de l'image et conversion en matrice numpy
    to_transform = np.array(Image.open(s))
    # standardisation de la luminosité
    to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
    # normalisation de la teinte
    transformed = normalizer.transform(to_transform)
    # affichage graphique
    fig, ax = plt.subplots(1, 3, figsize=(10, 3))
    ax[0].imshow(to_transform)
    ax[0].set_title("Image d'origine")
    ax[1].imshow(target_img)
    ax[1].set_title("Lame de référence")
    ax[2].imshow(transformed)
    ax[2].set_title("Image normalisée")
    fig.tight_layout()
    plt.show()

On peut également utiliser un patch plutôt qu'une lame pour estimer les paramètres de la méthode de normalisation.

In [ ]:
for s in sample(source_patches, 10):
    # sélection aléatoire d'un patch de référence
    target_patch = sample(target_patches, 1)[0] 
    # ouverture de l'image et conversion en matrice numpy
    to_transform = np.array(Image.open(s))
    # standardisation de la luminosité
    to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
    # normalisation de la teinte
    transformed = normalizer.transform(to_transform)
    # affichage graphique
    fig, ax = plt.subplots(1, 3, figsize=(8, 3))
    ax[0].imshow(to_transform)
    ax[0].set_title("Image d'origine");ax[0].axis('off')
    ax[1].imshow( np.array(Image.open(target_patch)))
    ax[1].set_title("Patch de référence");ax[1].axis('off')
    ax[2].imshow(transformed)
    ax[2].set_title("Image normalisée");ax[2].axis('off')
    fig.tight_layout()
    plt.show()

# Bac à sable

**Explorez les fonctions, les données et autres visualisation comme vous le souhaitez ! Vous pouvez également poser toutes vos questions sur la visio, je suis là pour ça :-)**


Si le code n'est pas votre tasse de thé, vous pouvez consulter ces articles (disponibles sur le Drive):
- https://www.nature.com/articles/s41416-020-01122-x (Echle et al., BJC, 2020): très bon article paru en 2020 si le potentiel du Deep Learning en pathologie digitale pour la découverte de nouveaux biomarqueurs 
- https://www.nature.com/articles/s41591-019-0462-y/ (Kather et al, Nat Med, 2019): un article pionier dans le domaine de la pathologie digitale accessible au néophyte
- https://www.nature.com/articles/srep27988 (Kather et al, Nat Med, 2016): l'article dont nous allons nous inspirer pour la suite de ce TP
- https://www.nature.com/articles/s41379-021-00919-2 (Nat Med, Baxi et al., 2021): une revue des méthodes publiées en pathologie digitale, applications, limites et implémentations dans le cadre courant du soin

In [ ]:
# copiez / collez ici certains éléments de code (ou pas si vous êtes à l'aise!)